# News Article Sentiment Classification

In this hands-on workshop, we'll practice classifying text on a slightly larger dataset.

Input: 8000 news articles that are labeled by relevance to the US Economy.

Task: Fit a model that classifies the articles based on whether each is relevant to the US Economy. This is a binary classification task.

Other uses of the same dataset (feel free to practice these on your owntime):
- Multi-class classification on positivity score
- Regression on relevance confidence, for the relevant articles

## Dataset

CSV: https://www.figure-eight.com/wp-content/uploads/2016/03/Full-Economic-News-DFE-839861.csv

Source: https://www.figure-eight.com/data-for-everyone/

Description:

>Contributors read snippets of news articles. They then noted if the article was relevant to the US economy and, if so, what the tone of the article was. Tone was judged on a 9 point scale (from 1 to 9, with 1 representing the most negativity). Dataset contains these judgments as well as the dates, source titles, and text. Dates range from 1951 to 2014.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Data Exploration (20 minutes)

1. Load the data into pandas
2. Check for NaNs (if any) and decide what you would do with them

In [ ]:
df = pd.read_csv('d:/tmp/news-article/Full-Economic-News-DFE-839861.csv', encoding='latin1',
                usecols=['relevance', 'text'])
df.head()

In [ ]:
df.drop(df.loc[df.relevance=='not sure'].index, inplace=True)
df.relevance.unique()

## Data Transformation (60 minutes)

1. Convert the relevance "yes/no" into numbers using LabelEncoder
2. Perform text feature extraction on the "text" column:

   a. Use spaCy to tokenize and lemmatize.
   
   b. Pass the tokenized text into TfidfVectorizer. You can decide to train with 1-gram or 2-gram (or more if your computer is faster)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['y'] = encoder.fit_transform(df.relevance)
df.head()

In [ ]:
import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

nlp = spacy.load('en_core_web_sm')
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

# transformation function to use in pd.DataFrame.apply()
def tokenize_text(text):
    """Tokenizes the text by lemmatizing and removing stop words
    Args:
        text - the input text
    Returns:
        a list of tokens
    """
    # process the text
    doc = nlp(text)
    
    # https://spacy.io/api/token
    lemmas = [lemmatizer(token.text, token.pos_) for token in doc
              if not token.is_stop and token.is_alpha]

    # create a string
    return ' '.join([item for sublist in lemmas for item in sublist])

In [ ]:
# Test
%time tokenize_text(df.text[0])

In [ ]:
# Tokenize the column (takes about 10 minutes)
# %time df_tokenized = df.text.apply(tokenize_text)

# Save the tokenized columns to CSV, so that we don't have to tokenize again
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html

# df['tokenized'] = df_tokenized
# df.to_csv('d:/tmp/news-article/Full-Economic-News-DFE-839861.tokenized.csv', encoding='latin1',
#          index=False)

In [2]:
df = pd.read_csv('d:/tmp/news-article/Full-Economic-News-DFE-839861.tokenized.csv', encoding='latin1')
df.head()

,relevance,text,y,tokenized
0,yes,NEW YORK -- Yields on most certificates of dep...,1,new york yield certificate deposit offer major...
1,no,The Wall Street Journal Online</br></br>The Mo...,0,the wall street journal the morning brief look...
2,no,WASHINGTON -- In an effort to achieve banking ...,0,washington in effort achieve banking reform se...
3,no,The statistics on the enormous costs of employ...,0,the statistic enormous cost employee drug abus...
4,yes,NEW YORK -- Indecision marked the dollar's ton...,1,new york indecision mark dollar tone trader pa...


In [3]:
df_tokenized = df.tokenized
df_tokenized.head()

0    new york yield certificate deposit offer major...
1    the wall street journal the morning brief look...
2    washington in effort achieve banking reform se...
3    the statistic enormous cost employee drug abus...
4    new york indecision mark dollar tone trader pa...
Name: tokenized, dtype: object

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Apply the vectorizer on all tokenized rows
vectorizer = TfidfVectorizer(lowercase=False, decode_error='ignore')

X = vectorizer.fit_transform(df_tokenized)

# print a few features
print(vectorizer.get_feature_names()[:10])

['aa', 'aaa', 'aacer', 'aadvantage', 'aagrf', 'aairl', 'aalrl', 'aame', 'aap', 'aaron']


In [5]:
# check the distribution of the relevant / irrelevant articles
df.groupby(['y']).size()

y
0    6571
1    1420
dtype: int64

## Visualize (20 minutes)

Use `TSNE` to visualize the Tfidf vectors into a 2-D plot or 1-D plot

In [ ]:
from sklearn.manifold import TSNE

# Sample a portion of the dataset for t-SNE (otherwise too slow)
density = 0.3

df_display = pd.DataFrame(X.todense())
df_display['y'] = df.y
sample = df_display.sample(frac=density, random_state=42)
sample_X = sample.iloc[:, :-1] # pick all but the last column ('y')

tsne = TSNE(n_components=2, random_state=42)
%time X_2d = tsne.fit_transform(sample_X)

print(X_2d.shape)

fig, ax = plt.subplots(figsize=(15, 10))
ax.scatter(X_2d[:, 0], X_2d[:, 1], c=sample.y)

ax.set(title='t-SNE plot for News Articles (X) and Relevance (y), density %.2f' % density,
       xlabel='X_2d[:, 0]', ylabel='X_2d[:, 1]')
ax.grid()
plt.show()

In [ ]:
tsne = TSNE(n_components=1, random_state=42)
%time X_1d = tsne.fit_transform(sample_X)

print(X_1d.shape)

fig, ax = plt.subplots(figsize=(15, 10))
ax.scatter(X_1d, sample.y)

ax.set(title='t-SNE 1D plot for News Articles (X) and Relevance (y), density %.2f' % density,
       xlabel='X_1d', ylabel='y')
ax.grid()
plt.show()

## Train (30 minutes)

1. Train/test split and shuffle the dataset
2. Train your favourite models with the dataset.  Keep in mind that KNeighbors, SVM and MLPClassifier can be slow.
3. Get classification_report metric

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sparse, df.y, random_state=42)

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report

classifiers = [
    DummyClassifier(random_state=42),
    LogisticRegression(random_state=42),
    SGDClassifier(tol=1e-3, max_iter=1000, random_state=42),
    GaussianNB(),
    DecisionTreeClassifier(random_state=42),
    MLPClassifier(random_state=42)
]

for clf in classifiers:
    print(type(clf))
    %time clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print(classification_report(y_test, pred))

In [ ]:
# Example of plotting the ROC curves to compare classifiers
from sklearn.metrics import roc_curve, auc

fig, (ax0, ax1) = plt.subplots(nrows=2, ncols=1, figsize=(20, 20))

#
# plot the dummy classifier
#
baseline = classifiers[0] # see list above
y_confidence_baseline = baseline.predict_proba(X_test)

# positive label = class 0
fpr0, tpr0, _ = roc_curve(y_test, y_confidence_baseline[:, 0], pos_label=0)
ax0.plot(fpr0, tpr0, label='Baseline (area = %f)' % auc(fpr0, tpr0), linestyle='dashed')

# positive label = class 1
fpr1, tpr1, _ = roc_curve(y_test, y_confidence_baseline[:, 1], pos_label=1)
ax1.plot(fpr1, tpr1, label='Baseline (area = %f)' % auc(fpr1, tpr1), linestyle='dashed')

#
# plot the balanced classifiers
#
for clf in balanced_classifiers:
    # some classifiers provide .decision_function(), others provide .predict_proba()
    if hasattr(clf, 'decision_function'):
        y_confidence = clf.decision_function(X_test)
    
        # positive label = class 0
        fpr0, tpr0, _ = roc_curve(y_test, y_confidence, pos_label=0)

        # positive label = class 1
        fpr1, tpr1, _ = roc_curve(y_test, y_confidence, pos_label=1)    
    else:
        y_confidence = clf.predict_proba(X_test)

        # positive label = class 0
        fpr0, tpr0, _ = roc_curve(y_test, y_confidence[:, 0], pos_label=0)

        # positive label = class 1
        fpr1, tpr1, _ = roc_curve(y_test, y_confidence[:, 1], pos_label=1)    

    ax1.plot(fpr1, tpr1, label='%s (area = %f)' % (clf.__class__, auc(fpr1, tpr1)))
    ax0.plot(fpr0, tpr0, label='%s (area = %f)' % (clf.__class__, auc(fpr0, tpr0)))        


# put the plots together
ax0.set(xlabel='false positive rate', ylabel='true positive rate',
       title='ROC curves (positive label="irrelevant")')
ax0.legend()
ax0.grid()

ax1.set(xlabel='false positive rate', ylabel='true positive rate',
       title='ROC curves (positive label="relevant")')
ax1.legend()
ax1.grid()

plt.show()

## Predictions (30 minutes)

1. Go to reuters.com or anywhere and find any news article.
2. Get the text of the article and pass it to your best performing model.

  a. You need to first tokenize and lemmatize your test text
  
  b. Apply `TfidfVectorizer` to get the test input vector
  
  c. Run `predict` to see if your model correctly classified the text as relevant to the US Economy

In [ ]:
sample_news = ["""A sell-off in Chinese markets knocked Asian stocks on Wednesday as U.S. threats of tariffs on an additional $200 billion worth of Chinese goods pushed the world’s two biggest economies ever closer to a full-scale trade war.
FILE PHOTO: People walk past an electronic board showing Japan's Nikkei average outside a brokerage in Tokyo, Japan, March 23, 2018. REUTERS/Toru Hanai
Washington proposed the extra tariffs after efforts to negotiate a solution to the dispute failed to reach an agreement, senior administration officials said on Tuesday.

The United States had just imposed tariffs on $34 billion worth of Chinese goods on Friday, drawing immediate retaliatory duties from Beijing on U.S. imports in the first shots of a heated trade war. U.S. President Donald Trump had warned then that his country may ultimately impose tariffs on more than $500 billion worth of Chinese imports - roughly the total amount of U.S. imports from China last year.

“With no early end appearing to be in sight for the escalating ‘tit-for-tat’ world trade frictions and rising trade protectionism, global trade wars have become one of the key downside risks to world growth and trade in the second half of 2018 and for 2019,” wrote Rajiv Biswas, Asia Pacific chief economist at IHS Markit.

MSCI’s broadest index of Asia-Pacific shares outside Japan fell 1.1 percent. The index had gained for the past two sessions, having enjoyed a lull from the trade war fears that lashed global markets last week."""
,
"""TOKYO (Reuters) - Japanese regulators on Wednesday said Apple Inc (AAPL.O) may have breached antitrust rules by forcing mobile service providers to sell its iPhones cheaply and charge higher monthly fees, denying consumers a fair choice.

FILE PHOTO: A member of Apple staff takes pictures as new iPhone X begins to sell at an Apple Store in Beijing, China November 3, 2017. REUTERS/Damir Sagolj/File Photo
The Fair Trade Commission (FTC) said that Apple had forced NTT Docomo Inc (9437.T) , KDDI Corp (9433.T) and SoftBank Group Corp (9984.T) to offer subsidies and sell iPhones at a discount.

Apple Inc
190.35
AAPL.ONASDAQ
-0.00(-0.00%)
AAPL.O
AAPL.O9437.T9433.T9984.T
The FTC, which began looking into Apple’s sales practices in 2016, did not punish Apple as the U.S. company had agreed to revise its contracts with the carriers, it said
TOKYO (Reuters) - Japanese regulators on Wednesday said Apple Inc (AAPL.O) may have breached antitrust rules by forcing mobile service providers to sell its iPhones cheaply and charge higher monthly fees, denying consumers a fair choice.

FILE PHOTO: A member of Apple staff takes pictures as new iPhone X begins to sell at an Apple Store in Beijing, China November 3, 2017. REUTERS/Damir Sagolj/File Photo
The Fair Trade Commission (FTC) said that Apple had forced NTT Docomo Inc (9437.T) , KDDI Corp (9433.T) and SoftBank Group Corp (9984.T) to offer subsidies and sell iPhones at a discount.

Apple Inc
190.35
AAPL.ONASDAQ
-0.00(-0.00%)
AAPL.O
AAPL.O9437.T9433.T9984.T
The FTC, which began looking into Apple’s sales practices in 2016, did not punish Apple as the U.S. company had agreed to revise its contracts with the carriers, it said
"""
]

df_sample_news = pd.DataFrame(data={'text': sample_news})

# first tokenize and lemmatize your test text
df_sample_news['tokenized_text'] = df_sample_news.text.apply(tokenize_text)
df_sample_news

In [ ]:
# Apply TfidfVectorizer to get the test input vector
sample_news = vectorizer.transform(df_sample_news.tokenized_text).todense()
print(sample_news.shape)

In [ ]:
# Run predict to see if your model correctly classified the text as relevant to the US Economy

for clf in balanced_classifiers:
    print(clf.__class__)
    predictions = clf.predict(sample_news)
    
    if hasattr(clf, 'predict_proba'):
        probabilities = clf.predict_proba(sample_news)
        for prediction, probability in (zip(predictions, probabilities)):
            print(prediction, probability)
    else:
        distances = clf.decision_function(sample_news)
        for prediction, distance in (zip(predictions, distances)):
            print(prediction, distance)

## Exploring Improvements 1 - Manually Balance Training Set

In [ ]:
# check the distribution of the relevant / irrelevant articles
df.groupby(['y']).size()

In [ ]:
# let's cut the irrelevant articles by 1/4
# Alternatively, you can also collect and label more relevant articles
df_irrelevant = df[df.y == 0].sample(frac=0.25, random_state=42)
df_irrelevant.shape

In [ ]:
# keep all the relevant articles
df_relevant = df[df.y == 1]
df_relevant.shape

In [ ]:
# make our balanced dataset
df_balanced = pd.concat([df_irrelevant, df_relevant], axis=0) # concat rows
df_balanced.groupby(['y']).size()

In [ ]:
df_balanced.columns

In [ ]:
# Featurize, fit, metrics, predict, ...

# featurize
X = vectorizer.fit_transform(df_balanced.tokenized)

# fit
X_train, X_test, y_train, y_test = train_test_split(X, df_balanced.y, random_state=42)
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

# metrics
print(classification_report(y_test, clf.predict(X_test)))

# predict
sample_news = vectorizer.transform(df_sample_news.tokenized_text).todense()
predictions = clf.predict(sample_news)
if hasattr(clf, 'predict_proba'):
    probabilities = clf.predict_proba(sample_news)
    for prediction, probability in (zip(predictions, probabilities)):
        print(prediction, probability)
else:
    distances = clf.decision_function(sample_news)
    for prediction, distance in (zip(predictions, distances)):
        print(prediction, distance)

## Exploring Improvements 2 - N-grams

In [ ]:
vectorizer_ngram = TfidfVectorizer(ngram_range=(2, 2))

In [ ]:
# Featurize, fit, metrics, predict, ...

# featurize
X = vectorizer_ngram.fit_transform(df_balanced_sampled.tokenized)

# fit
X_train, X_test, y_train, y_test = train_test_split(X, df_balanced_sampled.y, random_state=42)
clf = LogisticRegression()
clf.fit(X_train, y_train)

# metrics
print(classification_report(y_test, clf.predict(X_test)))

# predict
sample_news = vectorizer_ngram.transform(df_sample_news.tokenized_text).todense()
predictions = clf.predict(sample_news)
if hasattr(clf, 'predict_proba'):
    probabilities = clf.predict_proba(sample_news)
    for prediction, probability in (zip(predictions, probabilities)):
        print(prediction, probability)
else:
    distances = clf.decision_function(sample_news)
    for prediction, distance in (zip(predictions, distances)):
        print(prediction, distance)